Step 1) First  step is to  extract  data from pdf

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams, LTChar, LTTextBox
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal, LTAnno
from pdfminer.high_level import extract_pages
import json

def is_digit(char):
    return char.isdigit()

def extract_font_info(layout):
    max_font_size = 0  # Variable to store the largest font size

    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox):
            for text_line in lt_obj:
                for character in text_line:
                    if isinstance(character, LTChar):
                        current_char = character.get_text()

                        # Skip digit characters
                        if is_digit(current_char):
                            continue

                        current_font_size = character.size

                        # Update max_font_size if the current font size is larger
                        if current_font_size > max_font_size:
                            max_font_size = current_font_size

    return max_font_size

def font_size(path):
    with open(path, 'rb') as fp:
        parser = PDFParser(fp)
        document = PDFDocument(parser)

    # Check if the document allows text extraction. If not, abort.
    #if not document.is_extractable:
        #raise PDFTextExtractionNotAllowed

    # Create a PDF resource manager object that stores shared resources.
        rsrcmgr = PDFResourceManager()

    # Create a PDF device object.
        device = PDFDevice(rsrcmgr)

    # BEGIN LAYOUT ANALYSIS
    # Set parameters for analysis.
        laparams = LAParams()

    # Create a PDF page aggregator object.
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

    # Create a PDF interpreter object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        total_pages = len(list(PDFPage.create_pages(document)))
        start_page = int(total_pages * 0.5)
        end_page = int(total_pages * 0.6)

        largest_font_size = 0  # Variable to store the largest font size across all pages

        for page_number, page in enumerate(PDFPage.create_pages(document)):
            if start_page <= page_number < end_page:
                interpreter.process_page(page)
                layout = device.get_result()
                current_page_max_font_size = extract_font_info(layout)

            # Update largest_font_size if the current page's max font size is larger
                if current_page_max_font_size > largest_font_size:
                    largest_font_size = current_page_max_font_size

    return largest_font_size
def extract_pdf(path):
    temp_pdf = []
    temp_dict = {}
    page_data = ""  # Initialize page_data
    largest_font_size = font_size(path)

    for page_layout in extract_pages(path):
        for element in page_layout:      # extracting the pages
            if isinstance(element, LTTextContainer):
                for text_line in element: # extracting the lines in pages
                    if isinstance(text_line, LTTextLineHorizontal):
                        for chr in text_line:
                            if isinstance(chr, LTChar):
                            # Check for bold, italic, and hyperlink

                                if (chr.size >= largest_font_size):
                                    if len(page_data) > 0:
                                        temp_dict['data'] = page_data
                                        temp_pdf.append(temp_dict)
                                        temp_dict = {}
                                    temp_dict["title"] = text_line.get_text()
                                    page_data = ""
                                    has_content_started = True
                                    break
                                elif (chr.size < largest_font_size):
                                    page_data += text_line.get_text()
                                    break

    out_file = open("Save the path to save the json file.json", "w", encoding='utf-8')  # Add encoding='utf-8'
    json.dump(temp_pdf, out_file, indent=6, ensure_ascii=False)  # Add ensure_ascii=False
    out_file.close()




if __name__=="__main__":
    extract_pdf(r"Specify  your pdf file location")


Note:-
1) You have to  uplaod your  pdf location
2) What ever  data which  is Extrcated from  the pdf is json formate,so  please save data in  JSON  Foramte.

Step 2) Set up  groq model

!pip install langchain-groq groq

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
api_key="Put_Your_Groq_API_Key"

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=api_key, model="llama3-70b-8192")

Step 3) Now once again  specify  your  Input  location  file, just  for safer  side

In [ ]:
INPUT_PATH = "Specify  your JSON file path"


Step 4) Now the Question and Answer part  comes  in  play

In [ ]:
import os
import re
import json
from tqdm import tqdm


# INPUT_PATH = JSON_OUTPUT
SEPARATOR = '\\'
OUTPUT_FILE = f"{INPUT_PATH.rsplit(SEPARATOR, 1)[1].split('.')[0]}_qna.json"
OUTPUT_PATH = f"D:\\Pdf_Extraction\\{OUTPUT_FILE}"

CONTEXT_THRESHOLD_SHORT = 2200
CONTEXT_THRESHOLD_LONG = 8000
PARTITION_SIZE = 6000

def generate_system_prompt() -> str:
    """Generate the system prompt for the LLM."""
    return """
    Imagine you're a super-powered question master! Your mission: transform the context into a unique, captivating, relevant and thought-provoking Questions and Answers session. Here's the key to crafting exceptional questions:
        
        Language: All the questions and answers should only be generated in Marathi and no other language.
        Beyond the Obvious: Dive deeper than surface-level facts. Challenge users to analyze the text's meaning and underlying implications.
        Fresh Perspectives: Craft unique questions that avoid simply restating information from the passage. No repeats allowed! And Think outside the box!
        Laser Focus: Keep your sights on the core ideas. Don't get bogged down in minor details.
        Spark Curiosity: Design questions that ignite users' interest and encourage them to truly grasp the text's essence.
        Variety is Key: Mix up question types (factual, analytical, open-ended) to keep the Q&A session engaging.
        Challenge Accepted: Push users to think critically and develop their interpretations and connections.
        Real-World Relevance: Explore how the text connects to broader concepts and real-world applications.
        Prediction Power: Encourage users to predict potential outcomes or consequences based on the information presented.
        Creative Spark: For creative content, delve into the author's motivations, potential symbolism, and alternative interpretations.
        Go Deeper, Go Further: Don't stop at the first answer! Prompt followup questions to encourage deeper understanding and analysis.
    """

def generate_user_prompt(context: str) -> str:
    """Generate the user prompt for the LLM."""
    template = """
    context: {context}

    Q1:-
    A1:-

    Q2:-
    A2:-

    Q3:-
    A3:-

    Q4:-
    A4:-

    Q5:-
    A5:-

    Q6:-
    A6:-

    Q7:-
    A7:-

    Q8:-
    A8:-

    Q9:-
    A9:-

    Q10:-
    A10:-

    Q11:-
    A11:-

    Q12:-
    A12:-

    Q13:-
    A13:-

    Q14:-
    A14:-

    Q15:-
    A15:-
    """
    return template.format(context=context)

def generate_full_prompt(context: str) -> str:
    """Generate the full prompt for the LLM."""
    system_prompt = generate_system_prompt()
    user_prompt = generate_user_prompt(context)
    return f"{system_prompt}\n\n{user_prompt}"

def llm_invoke(prompt: str) -> str:
    """Invoke the LLM with the given prompt."""
    try:
        response = llm.invoke(prompt).content
        return response
    except Exception as e:
        print(f"Error invoking LLM: {e}")
        return None

def process_context(context: str) -> list:
    """Process the context and generate questions."""
    if len(context) > CONTEXT_THRESHOLD_LONG:
        print("<==========entered if block==========>")
        parts = [context[i : i + PARTITION_SIZE] for i in range(0, len(context), PARTITION_SIZE)]
        responses = []
        for part in parts:
            prompt = generate_full_prompt(part)
            response = llm_invoke(prompt)
            if response:
                responses.append(response)
        return responses
    elif len(context) < CONTEXT_THRESHOLD_SHORT:
        print("<==========entered elif block==========>")
        prompt = generate_full_prompt(context)
        response = llm_invoke(prompt)
        if response:
            return [response]
    else:
        print("<==========entered else block==========>")
        prompt = generate_full_prompt(context)
        response = llm_invoke(prompt)
        if response:
            return [response]

def main() -> None:
    """Main function to process the input data and generate questions."""
    generated_questions = []
    with open(INPUT_PATH, encoding='utf-8') as f:
        data = json.load(f)
        total_items = len(data[1:50])
        for item in tqdm(data[1:50], desc="Processing items", unit="items"):
            title = item['title']
            context = item['data']
            print(len(context))
            questions = process_context(context)
            generated_questions.append({"title": title, "context": context, "questions": questions})

    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    with open(OUTPUT_PATH, "w") as f:
        json.dump(generated_questions, f, indent=6)

    print(f"Data Extraction Completed and saved at {OUTPUT_PATH}")

if __name__ == '__main__':
    main()

Step 5) Use this below,code when you  get  data in  Encoded form,

In [ ]:
import json

# Load the JSON file content
FILE_PATH = OUTPUT_PATH
with open(FILE_PATH, "r", encoding="utf-8") as file:

    # Directly load the JSON content, no need for decoding
    data = json.load(file)

print(data)

# Function to pretty print the JSON data
def pretty_print_json(data):
    print(json.dumps(data, ensure_ascii=False, indent=4))

# Print the data to verify
pretty_print_json(data)

# Optional: Save the data to a new JSON file
SEPARATOR = "\\"
OUTPUT_FILE = f"{FILE_PATH.rsplit(SEPARATOR, 1)[1].split('.')[0]}_decoded.json"
DECODED_OUTPUT = f"{OUTPUT_FILE}"
with open(DECODED_OUTPUT, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Data decoding Completed and saved at {DECODED_OUTPUT}")

Step 6) Now to we have to  seprate Question and Answer,and save the data in CSV  file 

In [ ]:
import json
import pandas as pd

# Replace this with the actual path to your JSON file
JSON_INPUT_PATH = DECODED_OUTPUT
# Path to save the output CSV file
CSV_FILE_PATH = r"Ram ki kahani.csv"

try:
    # Open and load the JSON file
    with open(JSON_INPUT_PATH, encoding="utf-8") as file:
        data = json.load(file)

        questions = []
        answers = []

        # Iterate through each entry in the JSON data
        for entry in data:
            # Iterate through each question block in the entry
            for question_block in entry["questions"]:
                # Split the block into individual questions and answers
                temp = question_block.split("\n\n")
                for qna in temp:
                    qna_temp = qna.split("\n")
                    # Ensure both question and answer are present
                    if len(qna_temp) > 1:
                        questions.append(qna_temp[0])
                        answers.append(qna_temp[1])

        print("Number of questions:", len(questions))
        print("Number of answers:", len(answers))
        
        # Create and save the DataFrame if there are questions and answers
        if questions and answers:
            print("Creating DataFrame...")
            df = pd.DataFrame({'Questions': questions, 'Answers': answers})
            print("Saving DataFrame to CSV file...")
            df.to_csv(CSV_FILE_PATH, index=False)
            print("Data has been saved to", CSV_FILE_PATH)

        else:
            print("No data to save.")

except FileNotFoundError:
    print("Error: JSON file not found.")
except json.JSONDecodeError as e:
    print("Error loading JSON data:", str(e))
except Exception as e:
    print("An unexpected error occurred:", str(e))


Step 7) Now to  clean  your  CSV  File we used re Expression

In [ ]:
import pandas as pd
import re

#   Function to remove Q1, A1, **Q1**, **A1** patterns and also asterisks from text
def clean_text(text):
    # Define the patterns
    patterns = [r'Q\d+:', r'A\d+:', r'\*\*Q\d+\*\*:', r'\*\*A\d+\*\*:', r'\*']
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    return text.strip()

# Function to apply the clean_text function to each element in a row
def clean_row(row):
    return row.apply(clean_text)

NEW_CSV_FILE_PATH = "Cleaned_Ram_ki kahani.csv"

try:
    # Load the CSV file
    csv_data = pd.read_csv(CSV_FILE_PATH)

    # Apply the cleaning function to each cell in the DataFrame
    cleaned_csv_data = csv_data.apply(clean_row, axis=1)

    # Save the cleaned data back to a CSV file
    cleaned_csv_data.to_csv(NEW_CSV_FILE_PATH, index=False)
    print(f"Cleaned data saved to {NEW_CSV_FILE_PATH}")

except FileNotFoundError:
    print(f"The file at {CSV_FILE_PATH} was not found.")
except pd.errors.EmptyDataError:
    print("The file is empty.")
except Exception as e:
    print(f"An error occurred: {e}")
